In [1]:
import pandas as pd
from datasets import load_dataset
import pyarrow as pa

# 1. Quick Version Check
print(f"Pandas version: {pd.__version__}")
print(f"PyArrow version: {pa.__version__}")

# 2. Try loading the dataset again
try:
    dataset = load_dataset("json", data_files="train.jsonl", split="train")
    print("\n✅ Success! Dataset loaded.")
    print(f"Number of rows: {len(dataset)}")
    print("First row sample:", dataset[0])
except Exception as e:
    print(f"\n❌ Still hitting an error: {e}")

Pandas version: 2.3.3
PyArrow version: 22.0.0


Generating train split: 0 examples [00:00, ? examples/s]


✅ Success! Dataset loaded.
Number of rows: 46740
First row sample: {'input': 'Schema:\nTable JUALE_SpecialHanger_CL(Oid, SupportHeight, SupportHeight_UT, UseSpring, UseSpring_ShortValue, UseSpring_LongValue, UseColdInsulation, UseColdInsulation_ShortValue, UseColdInsulation_LongValue, SpringL, SpringL_UT, SpringH, SpringH_UT, SpringDia, SpringDia_UT, RodDia, RodDia_UT, ClampH, ClampH_UT, ClampT)\n\nQuestion:\nList all records from JUALE_SpecialHanger_CL.', 'output': 'SELECT * FROM dbo.[JUALE_SpecialHanger_CL];'}


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/deepseek-coder-1.3b-base" # Example model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [8]:
!pip install --upgrade pyarrow datasets transformers

In [7]:
from datasets import load_dataset

# Use "json" as the format; it handles both .json and .jsonl files
dataset = load_dataset("json", data_files="train.jsonl", split="train")

# Check if it loaded correctly
print(dataset[0])

{'input': 'Schema:\nTable JUALE_SpecialHanger_CL(Oid, SupportHeight, SupportHeight_UT, UseSpring, UseSpring_ShortValue, UseSpring_LongValue, UseColdInsulation, UseColdInsulation_ShortValue, UseColdInsulation_LongValue, SpringL, SpringL_UT, SpringH, SpringH_UT, SpringDia, SpringDia_UT, RodDia, RodDia_UT, ClampH, ClampH_UT, ClampT)\n\nQuestion:\nList all records from JUALE_SpecialHanger_CL.', 'output': 'SELECT * FROM dbo.[JUALE_SpecialHanger_CL];'}


In [6]:
import re
import json
from pathlib import Path

schema_path = "TEST.txt"   # upload your file to Colab and set this path
output_path = "train.jsonl"

text = Path(schema_path).read_text(encoding="utf-8", errors="ignore")

# --- 1. Split into view blocks ---
view_blocks = re.split(r"\bCREATE\s+VIEW\s+\[dbo\]\.\[", text, flags=re.IGNORECASE)
# first element before the first view, skip it
view_blocks = view_blocks[1:]

views = []

for block in view_blocks:
    # block starts with ViewName] ...
    m_name = re.match(r"([^\]]+)\]", block)
    if not m_name:
        continue
    view_name = m_name.group(1).strip()

    # try to extract column list between (...) before AS
    # handles CREATE VIEW [dbo].[X] ( [col1], [col2], ... ) AS
    m_cols = re.search(
        r"\(\s*(.*?)\)\s*AS",
        block,
        flags=re.IGNORECASE | re.DOTALL
    )
    columns = []
    if m_cols:
        cols_raw = m_cols.group(1)
        # each column usually like [ColName],
        for line in cols_raw.splitlines():
            line = line.strip()
            if not line:
                continue
            # remove trailing commas
            line = line.rstrip(",")
            # extract [ColName]
            m_col = re.search(r"\[([^\]]+)\]", line)
            if m_col:
                columns.append(m_col.group(1).strip())
    else:
        # no explicit column list, we will not create examples for this view
        columns = []

    if not columns:
        continue

    views.append({"name": view_name, "columns": columns})

print(f"Extracted {len(views)} views with explicit column lists")

# --- 2. Helper for generating questions/queries ---

def basic_examples_for_view(view):
    name = view["name"]
    cols = view["columns"]
    examples = []

    # limit number of columns in schema string for readability
    schema_cols = ", ".join(cols[:20])
    schema_str = f"Table {name}({schema_cols})"

    # Example 1: select all
    q1 = {
        "input": f"Schema:\n{schema_str}\n\nQuestion:\nList all records from {name}.",
        "output": f"SELECT * FROM dbo.[{name}];"
    }
    examples.append(q1)

    # Example 2: select a couple of columns
    if len(cols) >= 2:
        c1, c2 = cols[0], cols[1]
        q2 = {
            "input": f"Schema:\n{schema_str}\n\nQuestion:\nShow {c1} and {c2} for every row in {name}.",
            "output": f"SELECT [{c1}], [{c2}] FROM dbo.[{name}];"
        }
        examples.append(q2)

    # Example 3: simple filter on first column that looks numeric or id
    # (heuristic: name contains 'Id', 'Oid', 'Height', 'Length', 'Diameter', 'H', 'L')
    numeric_like = None
    for c in cols:
        lc = c.lower()
        if any(k in lc for k in ["id", "oid", "height", "length", "diameter", "radius", "thickness", "width"]):
            numeric_like = c
            break
    if numeric_like:
        q3 = {
            "input": f"Schema:\n{schema_str}\n\nQuestion:\nGet rows from {name} where {numeric_like} is greater than 100.",
            "output": f"SELECT * FROM dbo.[{name}] WHERE [{numeric_like}] > 100;"
        }
        examples.append(q3)

    # Example 4: filter on a *_ShortValue column if present
    short_col = next((c for c in cols if "shortvalue" in c.lower()), None)
    if short_col:
        q4 = {
            "input": f"Schema:\n{schema_str}\n\nQuestion:\nFind rows in {name} with {short_col} equal to 'Yes'.",
            "output": f"SELECT * FROM dbo.[{name}] WHERE [{short_col}] = 'Yes';"
        }
        examples.append(q4)

    # Example 5: top 10 order by first column
    q5 = {
        "input": f"Schema:\n{schema_str}\n\nQuestion:\nList the top 10 rows from {name}.",
        "output": f"SELECT TOP 10 * FROM dbo.[{name}];"
    }
    examples.append(q5)

    return examples

# --- 3. Generate dataset for all views ---

all_examples = []
for v in views:
    all_examples.extend(basic_examples_for_view(v))

print(f"Generated {len(all_examples)} training examples")

with open(output_path, "w", encoding="utf-8") as f:
    for ex in all_examples:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print(f"Wrote {output_path}")


Extracted 13967 views with explicit column lists
Generated 46740 training examples
Wrote train.jsonl


In [6]:
import json
import pandas as pd

# 1. Manually open and load the file
with open(file_path, 'r') as f:
    # If it's JSONL:
    data = [json.loads(line) for line in f]
    # If it's standard JSON, just use: data = json.load(f)

# 2. Flatten the nested structures
df = pd.json_normalize(data)

print(df.head())

NameError: name 'file_path' is not defined

In [15]:
def tokenize_function(example):
    # Try to find the 'prompt' and 'response' regardless of name
    # We look for common keys if 'instruction' isn't there
    keys = example.keys()

    # Logic to find the instruction/input
    instr_key = next((k for k in keys if k in ['instruction', 'input', 'question', 'prompt']), list(keys)[0])
    # Logic to find the output/response
    out_key = next((k for k in keys if k in ['output', 'response', 'answer', 'code'] and k != instr_key), list(keys)[1])

    text = f"### Instruction: {example[instr_key]}\n### Response: {example[out_key]}"

    tokenized = tokenizer(text, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Map the function
tokenized_dataset = dataset.map(tokenize_function, remove_columns=dataset.column_names)

Map:   0%|          | 0/46740 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./deepseek-results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    max_steps=50,
    report_to="none",
    save_strategy="no" # Faster training for testing
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
)

# This is it!
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
